# RaspberryPi Platform
This tutorial explain the functionality of the library, which realize the communication between the single board computer and the micro-controller. The communication is based on the serial interface (UART), if the nucleo micro-controller and the Raspberry Pi are connected, than have to appear an device file in the /dev folder. In the most cases this file is named ttyAMC0 (ttyACM?). If you want to control the robot, you need to transmit the commands over the serial interface. You can write a new library for it in other language, like C/C++ or you can use our implemented library in Python language.

## Message formate
Lets start with describing the message formate. The messages between the two board are always composed by two parts. The first part is the key word of the messages, which symbolizes the function of it. The second part of the messages contains the data or the information. You can see this format in the section below: 

\#KEYA:HELLO;12;23;;\r\n

The key part have to contains 4 characters, the length of the second part can be variate. In the above example the key part is the "KEYA" word and the other part contains "HELLO;12;23;", where each data are separated by character ";". The character ":" is used as separator between the two parts and the ";\n\r" symbolize the end of the message. List of the some important keys:
   * MCTL - to move the robot
   * BRAK - to brake the robot
   * SPLN - to control on the Bezier curve
   * PIDA - to activate the PID control
   * ENPB - to activate the encoder speed publisher

If you want to test the serial communication, you can use for it a simple serial monitor, where the baud rate have to be the same like on the micro-controller, in our case it is 460800 bit/s. If you send the next simple command ("#ENPB:1.0;;\\r\\n") to activate a encoder rotation speed publisher, which will periodically transmit the motor rotation speed. Or you can send another simple command to control the motor rotation speed and the steering angle:

\#MCTL:10.0;5.0;;\r\n

You can use the enumerated keys to control the micro-control states and the robot movement. For example the key "MTCL" is used to set the motor rotation speed and the steering angle, the second parameter is the steering angle in degrees, it's limited mechanically, the positive value match the right direction. The first parameter can adjust the rotation speed, it equals the PWM signal in percent. If the PID control is activated, than it's the robot forward speed in meter per second.
If you want to stop the robot, you can use command with the key "BRAK", which has a single parameter for the steering angle, in this case the H-bridge (VNH5019) is in the brake to GND state. You can activate or deactivate the pid control method applied on the motor by the command with "PIDA" key, this command has a single parameter ("1" or "0"). The key "SPLN" is used to control the robot on the Bezier's curve, in this case you need to add four points, the movement duration and the direction (forward or backward). If you give the right points and the robot was well configured, than it describes the predefined Bezier's curve.

## Serial handler
If you want to communicate the micro-controller, you can use the object of the SerialHandler class, which is defined in the SerialHandler python code file. The object create automatically a new thread for receiving and processing the messages from the micro-controller. In the first step you have to import the specified code file and other useful libraries:

In [1]:
import sys
sys.path.append('../')
from Rpi_Platform import SerialHandler
import threading
import time

After the import you can initialized the object by giving the communication port (win) or the device file (linux) and you have to start the read thread:

In [2]:
serialHandler = SerialHandler.SerialHandler("COM44")
serialHandler.startReadThread()

The serial handler object has a predefined member function for each types of commands, if you want to set the motor input signal to 20% and the steering angle to 5 deg, you can do by the following line:

In [3]:
serialHandler.sendMove(20.0,5.0)

True

If you want to stop the robot, you can use the next member function:

In [4]:
serialHandler.sendBrake(-5.0)

True

Each member function with the sending functionality return a boolean, which is true, when the input parameters was right and the message was sent to the micro-controller. After the message was processed by the micro-controller, it send a response to your single board computer. If you want to be sure, that your message wasn't corrupted by another transition process, you can create an event and wait the response from the micro-controller. The read thread automatically check the responses and if an event object is subscribed to a key word, than activate the event. It means, your thread will be in the waiting state, until the response arrives or the timeout expires. You need to create an event object and to subscribe the key word:

In [5]:
ev1 = threading.Event()
serialHandler.readThread.addWaiter("MCTL", ev1)

If you subscribed to the key word "MCTL", you can send a move command, where you check firstly the message format and the response. If you haven't received any response with the "MTCL" key word, it seems, that the connection was interrupt. In this case it's possible, the robot doesn't execute your command. 

In [6]:
sent = serialHandler.sendMove(20.0,-10.0)
if sent:
    isConfirmed = ev1.wait(timeout=1.0)
    ev1.clear()
    if(isConfirmed):
        print("\tMoving was confirmed!")
    else:
        raise ConnectionError('Response', 'Response was not received!')

else:
    print("Sending proble")

	Moving was confirmed!


The same thing you can do for braking the robot:

In [7]:
ev2 = threading.Event()
serialHandler.readThread.addWaiter("BRAK", ev2, print)
sent = serialHandler.sendBrake(0.0)
if sent:
    isConfirmed = ev2.wait(timeout=1.0)
    ev2.clear()
    if(isConfirmed):
        print("\tBraking was confirmed!")
    else:
        raise ConnectionError('Response', 'Response was not received!')
else:
    print("Sending problem")

ack	Braking was confirmed!



Now you can seen the subscribing method is applied in the different form, we added a new input parameter. This input parameter have to be a callback function, which must have a single parameter. When this parameter is set up and the read thread received a message with the key word, than it applies this callback function by giving the message content. In the example below, we gave the print function and when the micro-controller sent the "ack" message, it was printed to screen.

If you want to control the motor rotation speed, it's necessary to activate the pid controller, you can do that by sending a "PIDA" message, like this:

In [8]:
serialHandler.sendPidActivation(True)

True

If the controller is activated, than you need to send the robot forward moving speed, instead of PWM signal. You can see in the next line a simple example, where the robot is moving to forward with the constant speed for 3 second:

In [9]:
sent = serialHandler.sendMove(0.33,0.0)
if sent:
    isConfirmed = ev1.wait(timeout=1.0)
    ev1.clear()
    if(isConfirmed):
        print("\tMoving was confirmed!")
    else:
        raise ConnectionError('Response', 'Response was not received!')

else:
    print("Sending problem")
time.sleep(3.0)
sent = serialHandler.sendBrake(0.0)
if sent:
    isConfirmed = ev2.wait(timeout=1.0)
    ev2.clear()
    if(isConfirmed):
        print("\tBraking was confirmed!")
    else:
        raise ConnectionError('Response', 'Response was not received!')
else:
    print("Sending problem")

	Moving was confirmed!
ack	Braking was confirmed!



If you want to describe a curve, which can be represented by a Berzier's curve, you have to define four points and the movement duration. You can see a simple example on the picture below, where P1 is the start point and the P2 is the end point. 
<img src="images/bez.png" height="300" width="303"/>

In the next section you can the code for controlling the robot on Bezier's curve:

In [10]:
serialHandler.sendBezierCurve(1+1j,1.56+0.44j,1.56-0.44j,1.0-1.0j,6.0,True)

True

If you want to control the robot by using PWM signal, you can deactivate the pid controller with the next code line:

In [11]:
serialHandler.sendPidActivation(False)

True

And the finally you have to close the serial interface and the started thread by the next code line:

In [12]:
serialHandler.close()